In [ ]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("simulated_patient_level_df.csv", sep=";")

In [ ]:
# --- 1. Colonnes date ---
date_cols = ["first_doc_date", "last_doc_date"]
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# --- 2. Colonnes numériques ---
num_cols = [
    # identifiant interne éventuel
    "PATIENT_INT_ID",

    # texte / docs
    "n_documents",
    "n_visits",
    "total_words",
    "mean_words_per_doc",
    "max_words_doc",
    "total_sentences",
    "mean_avg_sentence_len_words",

    # HPO (nouvelles colonnes et alias)
    "n_hpo_full_list",
    "n_hpo_unique",
    "hpo_total_per_doc",
    "hpo_unique_per_doc",
    "hpo_per_sentence",
    "hpo_per_1k_words",
    # alias pour compatibilité si présents
    "total_hpo_mentions",
    "n_hpo_codes",

    # autres
    "n_document_types",
    "n_units",
    "n_authors",
    "patient_age_first",
    "patient_age_last",
    "followup_days",
    "docs_per_year",
    "n_variants",
]

for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# --- 3. Colonnes de listes (variants + HPO) : ast.literal_eval ---
variant_list_cols = [
    "transcript_from_ehop",
    "transcript",
    "transcript_no_version",
    "chr",
    "pos",
    "ref",
    "alt",
    "var_id",
    "transcript_nm",
    "gene_symbol",
    "gene_hgnc_id",
    "acmg_classification",
    "acmg_criteria",
    "clinvar_classification",
    "clinvar_review_status",
]

# nouvelles colonnes de listes HPO
hpo_list_cols = [
    "HPO_full_list",
    "HPO_unique_list",
]

list_cols = variant_list_cols + hpo_list_cols

def parse_list_or_keep(x):
    """
    - si x est déjà une liste -> on le garde
    - si x est NaN -> on renvoie une liste vide
    - si x est une str qui ressemble à une liste -> ast.literal_eval
    - sinon -> on le met dans une liste singleton
    """
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    if isinstance(x, str):
        s = x.strip()
        # chaîne comme "[]" -> liste vide
        if s == "" or s == "[]":
            return []
        try:
            val = ast.literal_eval(s)
            # parfois literal_eval renvoie un scalaire
            if isinstance(val, list):
                return val
            else:
                return [val]
        except (ValueError, SyntaxError):
            # ce n’est pas une représentation de liste -> on encapsule
            return [x]
    # autres types (int, float, etc.)
    return [x]

for col in list_cols:
    if col in df.columns:
        df[col] = df[col].apply(parse_list_or_keep)

# --- 4. Colonnes catégorielles / texte ---
cat_cols = [
    "PATIENT_ID_HEX",
    "patient_label",
]
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].astype("string")

# Petit check
print(df.dtypes)
print("\nExemple de ligne (listes variants + HPO) :")
cols_to_show = [c for c in list_cols if c in df.columns]
print(df.iloc[2][cols_to_show].to_dict())


In [ ]:
df["variant_info"] = np.where(
    df["n_variants"] > 0,
    "with_variant",
    "without_variant"
).astype("string")

CATEGORY_COL = "patient_label"      # ou "variant_info"


### Histogramme âge

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(
    data=df,
    x="patient_age_first",
    bins=20,
    hue=CATEGORY_COL,
    multiple="stack",      # ou "layer" / "dodge" selon ce que tu préfères
    kde=False
)
plt.title(f"Histogramme de l'âge à l'arrivée, par {CATEGORY_COL}")
plt.xlabel("Age au premier document produit au service de génétique")
plt.ylabel("Nombre de patients")
plt.legend(title=CATEGORY_COL)
plt.show()


### Moyenne doc

In [ ]:
plt.figure(figsize=(8,5))

violin_color = "#6baed6"
box_edge_color = "#000000"
box_face_color = "#ffffff"

# Violin par catégorie
sns.violinplot(
    data=df,
    x=CATEGORY_COL,
    y="n_documents",
    inner=None,
    color=violin_color,
    alpha=0.5
)

# Boxplot superposé
sns.boxplot(
    data=df,
    x=CATEGORY_COL,
    y="n_documents",
    width=0.2,
    fliersize=2,
    linewidth=1.2,
    boxprops=dict(facecolor=box_face_color, edgecolor=box_edge_color),
    medianprops=dict(color=box_edge_color),
    whiskerprops=dict(color=box_edge_color),
    capprops=dict(color=box_edge_color),
)

plt.title(f"Nombre de documents par patient, selon {CATEGORY_COL}")
plt.xlabel(CATEGORY_COL)
plt.ylabel("Nombre de documents")
plt.tight_layout()
plt.show()
